In [2]:
# Importamos las librerías necesarias
import dash
from dash import dcc 
from dash import html
import plotly.express as px
import pandas as pd
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.sampling import BayesianModelSampling
import pandas as pd
from pgmpy.estimators import MaximumLikelihoodEstimator
import random
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import numpy as np
from pgmpy.inference import VariableElimination
from sklearn.metrics import f1_score
from ucimlrepo import fetch_ucirepo

In [3]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
server = app.server


In [4]:
 # fetch dataset 
predict_students_dropout_and_academic_success = fetch_ucirepo(id=697) 
  
# data (as pandas dataframes) 
X = predict_students_dropout_and_academic_success.data.features 
y = predict_students_dropout_and_academic_success.data.targets 

data=pd.DataFrame.from_dict(predict_students_dropout_and_academic_success['data']['original'])
data

,Marital Status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,1,17,5,171,1,1,122.0,1,19,12,...,0,0,0,0,0.000000,0,10.8,1.4,1.74,Dropout
1,1,15,1,9254,1,1,160.0,1,1,3,...,0,6,6,6,13.666667,0,13.9,-0.3,0.79,Graduate
2,1,1,5,9070,1,1,122.0,1,37,37,...,0,6,0,0,0.000000,0,10.8,1.4,1.74,Dropout
3,1,17,2,9773,1,1,122.0,1,38,37,...,0,6,10,5,12.400000,0,9.4,-0.8,-3.12,Graduate
4,2,39,1,8014,0,1,100.0,1,37,38,...,0,6,6,6,13.000000,0,13.9,-0.3,0.79,Graduate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4419,1,1,6,9773,1,1,125.0,1,1,1,...,0,6,8,5,12.666667,0,15.5,2.8,-4.06,Graduate
4420,1,1,2,9773,1,1,120.0,105,1,1,...,0,6,6,2,11.000000,0,11.1,0.6,2.02,Dropout
4421,1,1,1,9500,1,1,154.0,1,37,37,...,0,8,9,1,13.500000,0,13.9,-0.3,0.79,Dropout
4422,1,1,1,9147,1,1,180.0,1,37,37,...,0,5,6,5,12.000000,0,9.4,-0.8,-3.12,Graduate


In [5]:
#Tratamiento de los datos

#Eliminar todas las observaciones donde el Target sea 'Enrolled'
data_mod = data[data['Target'] != 'Enrolled']

#Categorizar variable Age of Enrollment
bins = [0, 19, 21, 24, 31, 41, float('inf')]  # Límites de los rangos
labels = ['Rango 16-18', 'Rango 19-20', 'Rango 21-23', 'Rango 24-30', 'Rango 31-40', 'Rango >41'] 
data_mod['age_range'] = pd.cut(data_mod['Age at enrollment'], bins=bins, labels=labels, right=False)

#Categorizar variable Curricular units 1st sem (approved)
bins = [0, 3, 6, 11, float('inf')]  # Límites de los rangos
labels = ['Rango 0-2', 'Rango 3-5', 'Rango 6-10', 'Rango >10'] 
data_mod['approved_sem1_range'] = pd.cut(data_mod['Curricular units 1st sem (approved)'], bins=bins, labels=labels, right=False)

#Categorizar variable Curricular units 1st sem (grade)
bins = [0, 10,11, 12,13, 14,16,float('inf')]  # Límites de los rangos
labels = ['Rango 0-10', 'Rango 10-11','Rango 11-12', 'Rango 12-13', 'Rango 13-14','Rango 14-16', 'Rango >16'] 
data_mod['grade_sem1_range'] = pd.cut(data_mod['Curricular units 1st sem (grade)'], bins=bins, labels=labels, right=False)

#Categorizar variable Admission grade
bins = [0, 100,115, 130,145, 160,float('inf')]  # Límites de los rangos
labels = ['Rango 0-100', 'Rango 100-115','Rango 115-130', 'Rango 130-145', 'Rango 145-160', 'Rango >160'] 
data_mod['grade_admission_range'] = pd.cut(data_mod['Admission grade'], bins=bins, labels=labels, right=False)

# Verifica el resultado
data_mod

C:\Users\maria\AppData\Local\Temp\ipykernel_19748\4164262718.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\maria\AppData\Local\Temp\ipykernel_19748\4164262718.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\maria\AppData\Local\Temp\ipykernel_19748\4164262718.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

,Marital Status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target,age_range,approved_sem1_range,grade_sem1_range,grade_admission_range
0,1,17,5,171,1,1,122.0,1,19,12,...,0.000000,0,10.8,1.4,1.74,Dropout,Rango 19-20,Rango 0-2,Rango 0-10,Rango 115-130
1,1,15,1,9254,1,1,160.0,1,1,3,...,13.666667,0,13.9,-0.3,0.79,Graduate,Rango 19-20,Rango 6-10,Rango 14-16,Rango 130-145
2,1,1,5,9070,1,1,122.0,1,37,37,...,0.000000,0,10.8,1.4,1.74,Dropout,Rango 19-20,Rango 0-2,Rango 0-10,Rango 115-130
3,1,17,2,9773,1,1,122.0,1,38,37,...,12.400000,0,9.4,-0.8,-3.12,Graduate,Rango 19-20,Rango 6-10,Rango 13-14,Rango 115-130
4,2,39,1,8014,0,1,100.0,1,37,38,...,13.000000,0,13.9,-0.3,0.79,Graduate,Rango >41,Rango 3-5,Rango 12-13,Rango 130-145
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4419,1,1,6,9773,1,1,125.0,1,1,1,...,12.666667,0,15.5,2.8,-4.06,Graduate,Rango 19-20,Rango 3-5,Rango 13-14,Rango 115-130
4420,1,1,2,9773,1,1,120.0,105,1,1,...,11.000000,0,11.1,0.6,2.02,Dropout,Rango 16-18,Rango 6-10,Rango 12-13,Rango 115-130
4421,1,1,1,9500,1,1,154.0,1,37,37,...,13.500000,0,13.9,-0.3,0.79,Dropout,Rango 24-30,Rango 6-10,Rango 14-16,Rango 145-160
4422,1,1,1,9147,1,1,180.0,1,37,37,...,12.000000,0,9.4,-0.8,-3.12,Graduate,Rango 19-20,Rango 3-5,Rango 13-14,Rango 145-160


In [6]:
# Definir una semilla
random.seed(12232)

# Definir el modelo
model = BayesianNetwork([("Gender", "Target"),("Displaced","Target"),
                         ("Scholarship holder","Target"),("Debtor","Target"),("Tuition fees up to date","Target"),
                         ("age_range","Target"),("approved_sem1_range","Target"),("grade_admission_range","approved_sem1_range")])

variables_seleccionadas = ['Gender', 'Displaced', 'Scholarship holder', 'Debtor',
                           'Tuition fees up to date','age_range','approved_sem1_range','grade_admission_range','Target']
# Nos quedamos con el dataset únicamente de las variables seleccionadas
data_sel = data_mod[variables_seleccionadas]
data_sel

,Gender,Displaced,Scholarship holder,Debtor,Tuition fees up to date,age_range,approved_sem1_range,grade_admission_range,Target
0,1,1,0,0,1,Rango 19-20,Rango 0-2,Rango 115-130,Dropout
1,1,1,0,0,0,Rango 19-20,Rango 6-10,Rango 130-145,Graduate
2,1,1,0,0,0,Rango 19-20,Rango 0-2,Rango 115-130,Dropout
3,0,1,0,0,1,Rango 19-20,Rango 6-10,Rango 115-130,Graduate
4,0,0,0,0,1,Rango >41,Rango 3-5,Rango 130-145,Graduate
...,...,...,...,...,...,...,...,...,...
4419,1,0,0,0,1,Rango 19-20,Rango 3-5,Rango 115-130,Graduate
4420,0,1,0,1,0,Rango 16-18,Rango 6-10,Rango 115-130,Dropout
4421,0,1,1,0,1,Rango 24-30,Rango 6-10,Rango 145-160,Dropout
4422,0,1,1,0,1,Rango 19-20,Rango 3-5,Rango 145-160,Graduate


In [7]:
train, test = train_test_split(data_sel, test_size=0.2, random_state=98)

In [9]:
#Calcular los parametros> probabilidades condicionales mediante maxima verosimilitud

emv = MaximumLikelihoodEstimator(model=model, data=train)

model.fit(data=train, estimator = MaximumLikelihoodEstimator)

#Chequiar modelo
model.check_model()

True

# Organizamos la aplicación en Dash

In [10]:
app.layout =html.Div([
                # 1. Creamos el input para el rango de edad
                dcc.Dropdown(
                id='RangoEdad',
                options=[{'label': i, 'value': i} for i in data_sel['age_range'].unique()],
                value='Rango 19-20'
                )
                
    ])